In [1]:
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
#----------- Table styling ------------------
style_cell = {
    'fontFamily': 'Open Sans',
    'textAlign': 'center',
    'height': '30px',
    'padding': '10px 22px',
    'whiteSpace': 'inherit',
    'overflow': 'hidden',
                'textOverflow': 'ellipsis',
}
style_cell_conditional = [
    {
        'if': {'column_id': 'State'},
        'textAlign': 'left'
    },
]
style_header = {
    'fontWeight': 'bold',
    'backgroundColor': "#3D9970",
    'fontSize': '16px'
}
style_data_conditional = [
    {
        'if': {'row_index': 'odd'},
        'backgroundColor': 'rgb(248, 248, 248)'
    }]
style_table = {
    'maxHeight': '70ex',
    'overflowY': 'scroll',
    'width': '100%',
    'minWidth': '100%',
}

In [3]:
#---------------- Load datasets -----------
df = pd.read_csv('./../data/external/global_data.csv')
df1 = pd.read_csv('./../data/external/SA_last20days.csv')
df2 = pd.read_csv('./../data/external/SA_from_day1.csv')
df4 = pd.read_csv('./../data/external/SA_3_daily_update.csv')
df5 = pd.read_csv('./../data/external/Covid_data.csv')
df6 = pd.read_csv('./../data/external/covid.csv',delimiter=',')
df3 = pd.read_csv('./../data/external/Sa_live.csv')
df_day =pd.read_csv('./../data/external/provincial_cumulative.csv')
# Commulative death per province 
df_death = pd.read_csv('./../data/external/provincial_death.csv')

In [4]:
df.head()

,Unnamed: 0,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,0,Afghanistan,AF,afghanistan,581,7653,5,178,49,850,2020-05-20T11:05:03Z
1,1,Albania,AL,albania,1,949,0,31,15,742,2020-05-20T11:05:03Z
2,2,Algeria,DZ,algeria,176,7377,6,561,121,3746,2020-05-20T11:05:03Z
3,3,Andorra,AD,andorra,0,761,0,51,4,628,2020-05-20T11:05:03Z
4,4,Angola,AO,angola,2,52,0,3,0,17,2020-05-20T11:05:03Z


In [5]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()
diff = tot_cases - nw_cases
diff2 = tot_recover - new_recover
diff3 = tot_deaths - new_deaths 

#Separating columns

#Corona  cases: Total and New
df_tot_cases = df.loc[:,['Country','TotalConfirmed']].sort_values(by='TotalConfirmed', ascending=False)
df_new_cases = df.loc[:,['Country','NewConfirmed']]

#Death Cases: Total and New 
df_tot_death = df.loc[:,['Country','TotalDeaths']]
df_new_death = df.loc[:,['Country','NewDeaths']]

#Recovery cases: Total and new
df_tot_recover = df.loc[:,['Country','TotalRecovered']]
df_new_recover = df.loc[:,['Country','NewRecovered']]
                      
#Sorting cases by total                     
df_cases = df_tot_cases.merge(df_new_cases, on='Country').sort_values(by=['TotalConfirmed'],
                                                                      ascending=False)
df_death = df_tot_death.merge(df_new_death, on='Country').sort_values(by=['TotalDeaths'],
                                                                      ascending=False)
df_recover = df_tot_recover.merge(df_new_recover, on='Country').sort_values(by=['TotalRecovered'], 
                                                                            ascending=False)


In [6]:
# a = df.TotalConfirmed.sum()-df.NewConfirmed.sum()

In [7]:
# total cases indicator
fig1 = go.Figure(go.Indicator(
    
    value = tot_cases,
    delta = {'reference': diff},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig1 = fig1.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Cases"},
        'mode' : "number+delta+gauge",
        }]
                         }})
fig1.show()

In [8]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()
diff = tot_cases - nw_cases

#Recoveries indicator
fig2 = go.Figure(go.Indicator(
    mode = "number+delta",
    value = tot_recover,
    delta = {'reference': diff2},
    domain = {'row': 0, 'column': 1}))
fig2 = fig2.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Recoveries"}
        }]
                         }})
# fig2.show()

In [9]:
# total deaths indicator
fig3 = go.Figure(go.Indicator(
    
    value = tot_deaths,
    delta = {'reference': diff3},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig3 = fig3.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Deaths"},
        'mode' : "number+delta+gauge",
        }]
                         }})
# fig3.show()

In [10]:
#----- Feature Creation from existing data ----
df['Active Cases'] = df['TotalConfirmed'] - df['TotalRecovered'] - df['TotalDeaths']
df['Closed Cases'] = df['TotalRecovered'] + df['TotalDeaths']
df['Recovery Rate'] = (df['TotalRecovered'] / df['TotalConfirmed'])*100
df['Case Fatality Rate'] = (df['TotalDeaths'] / df['TotalConfirmed']) * 100
df.head()

,Unnamed: 0,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Active Cases,Closed Cases,Recovery Rate,Case Fatality Rate
0,0,Afghanistan,AF,afghanistan,581,7653,5,178,49,850,2020-05-20T11:05:03Z,6625,1028,11.106756,2.325885
1,1,Albania,AL,albania,1,949,0,31,15,742,2020-05-20T11:05:03Z,176,773,78.187566,3.266596
2,2,Algeria,DZ,algeria,176,7377,6,561,121,3746,2020-05-20T11:05:03Z,3070,4307,50.779450,7.604717
3,3,Andorra,AD,andorra,0,761,0,51,4,628,2020-05-20T11:05:03Z,82,679,82.522996,6.701708
4,4,Angola,AO,angola,2,52,0,3,0,17,2020-05-20T11:05:03Z,32,20,32.692308,5.769231


In [11]:

df6.head()


,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,16/04/2020,16,4,2020,70,2,Afghanistan,AF,AFG,37172386.0
1,15/04/2020,15,4,2020,49,2,Afghanistan,AF,AFG,37172386.0
2,14/04/2020,14,4,2020,58,3,Afghanistan,AF,AFG,37172386.0
3,13/04/2020,13,4,2020,52,0,Afghanistan,AF,AFG,37172386.0
4,12/04/2020,12,4,2020,34,3,Afghanistan,AF,AFG,37172386.0


In [12]:
#SA commulative data graph
df_rsa = df6[df6.countriesAndTerritories == 'South_Africa'].reset_index()
fig_line = go.Figure()
fig_line.add_trace(go.Scatter(x = df4.timestamp,y= df4.active, name = 'Active Cases in SA'))
fig_line.update_layout(title = 'Commulative confirmed cases in SA as 17/03/2020')
fig_line.show()

In [13]:
df_rt = pd.read_csv('./../data/external//data-CiK32.csv')

fig_rt = go.Figure()
fig_rt.add_trace(
    go.Scatter(
        y=df_rt['Possible low Rt'],
        x=df_rt.date,
        name= 'Possible low Rt'
    ))
fig_rt.add_trace(
    go.Scatter(
        y=df_rt['Most likely Rt'],
        x=df_rt.date,
       name= 'Most likely Rt'
    ))
fig_rt.add_trace(
    go.Scatter(
        y=df_rt['Possible High Rt'],
        x=df_rt.date,
        name= 'Possible High Rt'
    ))

fig_rt.update_layout(hovermode='x',
                     xaxis=dict(
                         rangeslider=dict(
                             visible = True),
        type='date'
    ))



In [14]:
#Drop unwanted columns
# Provincial cases dataset
df_day = df_day.drop(['source','Unnamed: 0','YYYYMMDD'],axis = 1)
#append dates with missing data with previous day data(bc commulative)
df_day = df_day.fillna(method='ffill')

# df_death = df_death.drop(['source','Unnamed: 0','YYYYMMDD'],axis = 1)
#append dates with missing data with previous day data(bc commulative)
df_death = df_death.fillna(method='ffill')

#Commulative cases by province


fig_comm = go.Figure()
fig_comm.add_trace(
    go.Scatter(
        y=df_day['EC'],
        x=df_day.date,
        name= 'EC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['WC'],
        x=df_day.date,
        name= 'WC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['FS'],
        x=df_day.date,
        name= 'FS'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['GP'],
        x=df_day.date,
        name= 'GP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['KZN'],
        x=df_day.date,
        name= "KZN"
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['NW'],
        x=df_day.date,
        name= 'NW'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['LP'],
        x=df_day.date,
        name= 'LP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['MP'],
        x=df_day.date,
        name= 'MP'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['NC'],
        x=df_day.date,
        name= 'NC'
    ))
fig_comm.add_trace(
    go.Scatter(
        y=df_day['UNKNOWN'],
        x=df_day.date,
        name='Unlocated'
    ))
# fig_comm.add_trace(
#     go.Scatter(
#         y=df_day['total'],
#         x=df_day.date,
#         name='Total Confirmed in SA'
#     ))

fig_comm.update_layout(hovermode='x',
                       title = 'Commulative confirmed cases by province',
                       xaxis=dict(
                         rangeslider=dict(
                             visible = True),
                           
       
                       
    ))





In [15]:
#Total confirmed world map
data = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['TotalConfirmed'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(155,155,155)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Number of cases'
 )
 ) ]
layout = dict(
 autosize=True,
margin={"r":0,"t":0,"l":0,"b":0}
    
)
fig = go.Figure(data = data, layout = layout)

# Active cases world map

data1 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['Active Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(255,255,255)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Active cases'
 )
 ) ]
layout = dict(
 autosize=True,
 margin={"r":0,"t":0,"l":0,"b":0}   
)




fig1_ = go.Figure(data = data1, layout = layout)

#Closed cases world map
data2 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Closed Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(245, 197, 66)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Closed Cases'
 )
 ) ]
layout = dict(
 autosize=True,
 margin={"r":0,"t":0,"l":0,"b":0}   
)
fig2_ = go.Figure(data = data2, layout = layout)

#Recovery Rate World Map

data3 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Recovery Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(455,455,455)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Recovery Rate (%)'
 )
 ) ]
layout = dict(
 autosize=True,
 margin={"r":0,"t":0,"l":0,"b":0}   
)
fig3_ = go.Figure(data = data3, layout = layout)


#Case fatality rate world map

data4 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Case Fatality Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(555,555,555)',
 width = 10,
 )  ),
 colorbar = dict(
 title = 'Case Fatality Rate (%)'
 )
 ) ]
layout = dict(
    autosize=True,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig4_ = go.Figure(data = data4, layout = layout)
fig4_.show()


In [16]:


fig_prov_stacked = go.Figure(data=[
    go.Bar( name='GP',x=df_day.index, y=df_day.GP),
    go.Bar( name='WC',x=df_day.index, y=df_day.WC),
    go.Bar( name='KZN',x=df_day.index, y=df_day.KZN),
    go.Bar( name='EC',x=df_day.index, y=df_day.EC),
    go.Bar( name='NW',x=df_day.index, y=df_day.NW),
    go.Bar( name='NC',x=df_day.index, y=df_day.NC),
    go.Bar( name='MP',x=df_day.index, y=df_day.MP),
    go.Bar( name='LP',x=df_day.index, y=df_day.LP),
    go.Bar( name='FS',x=df_day.index, y=df_day.FS),
])
fig_prov_stacked.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="ALL",
                     method="update",
                     args=[{"visible": [True,True,True,True,True,True,True,True,True,]},
                           {"title": "Cumulative Confirmed Cases: Eastern Cape",
                            }]),
                dict(label="GP",
                     method="update",
                     args=[{"visible":  [True, False, False,False, False, False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Eastern Cape",
                            }]),
                dict(label="WC",
                     method="update",
                     args=[{"visible": [False,True,False,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Western Cape",
                          }]),
                dict(label="KZN",
                     method="update",
                     args=[{"visible": [False,False,True,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Free State",
                          }]),
                dict(label="EC",
                     method="update",
                     args=[{"visible": [False,False,False,True,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Gauteng",
                          }]),
                dict(label="NW",
                     method="update",
                     args=[{"visible": [False,False,False,False,True, False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Kwa Zulu Natal",
                          }]),
                dict(label="NC",
                     method="update",
                     args=[{"visible": [False,False,False,False,False,True,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: North West",
                          }]),
                dict(label="MP",
                     method="update",
                     args=[{"visible": [ False,False,False,False, False,False, True, False, False]},
                           {"title": "Cumulative Confirmed Cases: Limpopo",
                          }]),
                dict(label="LP",
                     method="update",
                     args=[{"visible": [ False,False,False,False, False,False,False,True,   False]},
                           {"title": "Cumulative Confirmed Cases: Mpumalanga",
                          }]),
                 dict(label="FS",
                     method="update",
                     args=[{"visible": [ False,False,False,False, False,False,False, False,True]},
                           {"title": "Cumulative Confirmed Cases: Unlocated",
                          }]),
            ]),
        )
    ])
# Change the bar mode
fig_prov_stacked.update_layout(barmode='stack',xaxis=dict(
    rangeslider=dict(
        visible = True)))
fig_prov_stacked.update_layout( title_text="Confirmed Cases by provinces",
                 xaxis_title="Days since the first confirmed case",
                  yaxis_title="Linear")
                            
fig_prov_stacked.show()

In [17]:
#-----------------Provinces commulative stacked bar graph-------------------------

# Initialize figure
fig_prov_log = go.Figure()

# Add Traces

fig_prov_log.add_trace(
    go.Scatter(
        y=[2,312],
        x=[2,10],
        line=dict(color='grey',
                  width=4, dash='dot'),
        name = '1st Day'
     
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=[2,59],
        x=[3,60],
        line=dict(color='grey',
                  width=4, dash='dot'),
        name = '2nd Day'
     
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=[2,36],
        x=[4,60],
        line=dict(color='grey',
                  width=4, dash='dot'),
        name = '3rd Day'
     
    ))


fig_prov_log.add_annotation(
            x=10,
            y=2.5,
            text="Cases Doubles every day",
          
)
fig_prov_log.add_annotation(
            x=21,
            y=.8,
            text="Doubles every 2nd day",
          
)

fig_prov_log.add_annotation(
            x=40,
            y=1.1,
            text="3rd day",
          
)


fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['EC'],
        x=df_day.index,
        name= 'EC'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['WC'],
        x=df_day.index,
        name= 'WC'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['FS'],
        x=df_day.index,
        name= 'FS'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['GP'],
        x=df_day.index,
        name= 'GP'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['KZN'],
        x=df_day.index,
        name= "KZN"
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['NW'],
        x=df_day.index,
        name= 'NW'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['LP'],
        x=df_day.index,
        name= 'LP'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['MP'],
        x=df_day.index,
        name= 'MP'
    ))
fig_prov_log.add_trace(
    go.Scatter(
        y=df_day['NC'],
        x=df_day.index,
        name= 'NC'
    ))
# fig_prov_log.add_trace(
#     go.Scatter(
#         y=df_day['UNKNOWN'],
#         x=df_day.index,
#         name='Unlocated'
#     ))



fig_prov_log.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="ALL",
                     method="update",
                     args=[{"visible": [True,True,True,True,True,True,True,True,True,True,True,True,True,]},
                           {"title": "Cumulative Confirmed Cases: Eastern Cape",
                            }]),
                dict(label="EC",
                     method="update",
                     args=[{"visible": [True,True,True,True, False, False, False,False, False, False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Eastern Cape",
                            }]),
                dict(label="WC",
                     method="update",
                     args=[{"visible": [True,True,True,False,True,False,False,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Western Cape",
                          }]),
                dict(label="FS",
                     method="update",
                     args=[{"visible": [True,True,True,False,False,True,False,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Free State",
                          }]),
                dict(label="GP",
                     method="update",
                     args=[{"visible": [True,True,True,False,False,False,True,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Gauteng",
                          }]),
                dict(label="KZN",
                     method="update",
                     args=[{"visible": [True,True,True,False,False,False,False,True, False,False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Kwa Zulu Natal",
                          }]),
                dict(label="NW",
                     method="update",
                     args=[{"visible": [True,True,True,False,False,False,False,False,True,False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: North West",
                          }]),
                dict(label="LP",
                     method="update",
                     args=[{"visible": [True,True,True,False, False,False,False,False, False,True, False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Limpopo",
                          }]),
                dict(label="MP",
                     method="update",
                     args=[{"visible": [True,True,True,False, False,False,False,False, False,False,True,  False, False]},
                           {"title": "Cumulative Confirmed Cases: Mpumalanga",
                          }]),
                dict(label="NC",
                     method="update",
                     args=[{"visible": [True,True,True,False, False,False,False,False, False,False,  False,True, False]},
                           {"title": "Cumulative Confirmed Cases: Northern Cape",
                          }]),
                 dict(label="Cumulative Confirmed Cases: UNKWNOWN",
                     method="update",
                     args=[{"visible": [True,True,True,False, False,False,False,False, False,False,False, False,True]},
                           {"title": "Unlocated",
                          }]),
            ]),
        )
    ])

# Set title
fig_prov_log.update_layout(title_text="Confirmed Cases by Province",
                 xaxis_title="Days since the first confirmed case",
                 yaxis_title="Log",
                 yaxis_type="log")

fig_prov_log.show()



In [18]:
a = df_day.set_index('date')
a = a.diff()
a = a.dropna()
a = a.reset_index()
df_day = a

# Initialize figure
fig_flat = go.Figure()

# Add Traces

fig_flat.add_trace(
    go.Scatter(
        y=[0,800],
        x=[18,18],
        line=dict(color='grey',
                  width=4, dash='dot'),
        name = 'lockdown'
     
    ))
fig_flat.add_trace(
    go.Scatter(
        y=[0,800],
        x=[55,55],
        line=dict(color='grey',
                  width=4, dash='dot'),
        name = 'level_4'
     
    ))


fig_flat.add_annotation(
            x=18,
            y=400,
            text="Lockdown starts",
          
)
fig_flat.add_annotation(
            x=55,
            y=500,
            text="Level 4",
          
)


          


fig_flat.add_trace(
    go.Scatter(
        y=df_day['EC'],
        x=df_day.index,
        name= 'EC'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['WC'],
        x=df_day.index,
        name= 'WC'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['FS'],
        x=df_day.index,
        name= 'FS'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['GP'],
        x=df_day.index,
        name= 'GP'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['KZN'],
        x=df_day.index,
        name= "KZN"
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['NW'],
        x=df_day.index,
        name= 'NW'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['LP'],
        x=df_day.index,
        name= 'LP'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['MP'],
        x=df_day.index,
        name= 'MP'
    ))
fig_flat.add_trace(
    go.Scatter(
        y=df_day['NC'],
        x=df_day.index,
        name= 'NC'
    ))
# fig.add_trace(
#     go.Scatter(
#         y=df_day['UNKNOWN'],
#         x=df_day.index,
#         name='Unlocated'
#     ))



fig_flat.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="ALL",
                     method="update",
                     args=[{"visible": [True,True,True,True,True,True,True,True,True,True,True,True,]},
                           {"title": "Cumulative Confirmed Cases: Eastern Cape",
                            }]),
                dict(label="EC",
                     method="update",
                     args=[{"visible": [True,True,True, False, False, False,False, False, False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Eastern Cape",
                            }]),
                dict(label="WC",
                     method="update",
                     args=[{"visible": [True,True,False,True,False,False,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Western Cape",
                          }]),
                dict(label="FS",
                     method="update",
                     args=[{"visible": [True,True,False,False,True,False,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Free State",
                          }]),
                dict(label="GP",
                     method="update",
                     args=[{"visible": [True,True,False,False,False,True,False,False,False,False,False,False]},
                           {"title": "Cumulative Confirmed Cases: Gauteng",
                          }]),
                dict(label="KZN",
                     method="update",
                     args=[{"visible": [True,True,False,False,False,False,True, False,False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Kwa Zulu Natal",
                          }]),
                dict(label="NW",
                     method="update",
                     args=[{"visible": [True,True,False,False,False,False,False,True,False,False, False, False]},
                           {"title": "Cumulative Confirmed Cases: North West",
                          }]),
                dict(label="LP",
                     method="update",
                     args=[{"visible": [True,True,False, False,False,False,False, False,True, False, False, False]},
                           {"title": "Cumulative Confirmed Cases: Limpopo",
                          }]),
                dict(label="MP",
                     method="update",
                     args=[{"visible": [True,True,False, False,False,False,False, False,False,True,  False, False]},
                           {"title": "Cumulative Confirmed Cases: Mpumalanga",
                          }]),
                dict(label="NC",
                     method="update",
                     args=[{"visible": [True,True,False, False,False,False,False, False,False,  False,True, False]},
                           {"title": "Cumulative Confirmed Cases: Northern Cape",
                          }]),
                 dict(label="Cumulative Confirmed Cases: UNKWNOWN",
                     method="update",
                     args=[{"visible": [True,True,False, False,False,False,False, False,False,False, False,True]},
                           {"title": "Unlocated",
                          }]),
            ]),
        )
    ])

# Set title
fig_flat.update_layout(title_text="Confirmed Cases by Province",
                 xaxis_title="Days since the first confirmed case",
                 yaxis_title="Log",
                 )

fig_flat.show()



In [19]:
a

,date,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total
0,07-03-2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,08-03-2020,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,09-03-2020,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,11-03-2020,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0
4,12-03-2020,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
68,15-05-2020,93.0,6.0,75.0,38.0,2.0,0.0,3.0,5.0,563.0,0.0,785.0
69,16-05-2020,150.0,2.0,52.0,16.0,2.0,1.0,1.0,1.0,606.0,0.0,831.0
70,17-05-2020,124.0,5.0,67.0,45.0,18.0,3.0,2.0,6.0,890.0,0.0,1160.0
71,18-05-2020,116.0,5.0,14.0,24.0,11.0,5.0,0.0,2.0,741.0,0.0,918.0


In [20]:
#Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# ------------- App -------------------------------
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config['suppress_callback_exceptions'] = True
app.layout = html.Div(children=[
        #header
    
    html.Div([
        html.Div([
           
        ], className="four columns"),
        

        html.Div([
            html.H3('Latest Covid 19 Update', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('#StayAtHome', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'right',
                            }),
            
            
        ], className="four columns")
    ], className="row"),
    
    dcc.Tabs(id='tabs-global', value='tab', children=[
        dcc.Tab(label='Global Map', value='tab-1'),
        dcc.Tab(label='South African cases', value='tab-2'),
        dcc.Tab(label='African cases', value='tab-3'),
    ]),
    html.Div(id='tabs-global-content')
])

In [21]:

#----------- Main tab call back ----
@app.callback(Output('tabs-global-content', 'children'),
              [Input('tabs-global', 'value')])
def render_content(tab):

    if tab == 'tab-1':
        return html.Div([
            html.Div([

                # VISUALISATIONS

                html.Div([
                      dcc.Graph(
                          id='tot_cases',
                            figure=go.Figure(fig1)
                          
                        ),
                html.Div([
                    dash_table.DataTable(
                            id='country_cases',
                            data=df_tot_cases.to_dict("rows"),
                            columns=[{"name": i, "id": i}
                                     for i in df_tot_cases.columns],
                            style_table=style_table,
                            style_cell=style_cell,
                            style_data_conditional=style_data_conditional,
                            style_header=style_header,
                            style_cell_conditional=style_cell_conditional,
                            filter_action="native"
                        )
                ])
                    
            ], className="row"),

                ], className="three columns"),
            
            html.Div([
                    dcc.Graph(
                          id='sa_cases',
                            figure=go.Figure(fig2)
                        ),
                    html.Div([
                        dcc.Tabs(id='tabs-globals', value='tabs', children=[
                            dcc.Tab(label='Total Cases', value='tab-1.'),
                            dcc.Tab(label='Active Cases', value='tab-2.'),
                            dcc.Tab(label='Solved Cases', value='tab-3.'),
                            dcc.Tab(label='Recovery Rate', value='tab-4.'),
                            dcc.Tab(label='Case Fatality Rate', value='tab-5.')
                        ], colors={
                                "border": "white",
                                "primary": "gold",
                                "background": "cornsilk"
                            }),
                     html.Div(id='tabs-globals-contents'),
            ], className="row"),
                ], className="six columns"),

                html.Div([
                     dcc.Graph(
                          id='indicator_tot_death',
                            figure=go.Figure(fig3)
                        ),


                    html.Div([
                        dcc.Graph(
                          id='sa_active',
                            figure=go.Figure(fig_line)
                        ),
                            
                       
            ], className="row")

                ], className="three columns"),
            ], className="row")

            # fOOTER
        html.Div([
                html.Div([
                    html.H3('Footer'),

                ], className="six columns"),

                html.Div([
                    html.H3('Footer'),

                ], className="six columns"),
            ], className="row")
  

    elif tab == 'tab-2':
        return html.Div([
            
            dcc.Graph(
                id='figure',
                figure=go.Figure(fig_prov_stacked)
            ),html.Div([
                dcc.Graph(
                    id='figure',
                    figure=go.Figure(fig_prov_log)
                ),html.Div([
                dcc.Graph(
                    id='figure',
                    figure=go.Figure(fig_flat)
                ),            
            ], className="row"),
                ], className="twelve columns")])
    else:
        return html.H1('African cases!!!')
    
#---- Small tab call back -----

@app.callback(Output('tabs-globals-contents', 'children'),
              [Input('tabs-globals', 'value')])
def render_content(tabs):
    
    if tabs == 'tab-1.':
        return dcc.Graph(

            id='figure',
            figure=
              go.Figure(fig)
            )
    elif tabs == 'tab-2.':   
        return dcc.Graph(

            id='figure',
            figure=
              go.Figure(fig1_)
            )
           
    elif tabs == 'tab-3.':   
        return dcc.Graph(

            id='figure',
            figure=
              go.Figure(fig2_)
            )
    elif tabs == 'tab-4.':
         return dcc.Graph(
             id='figure',
             figure=go.Figure(fig3_)
            )
    else:
        return dcc.Graph(

            id='figure',
            figure=
              go.Figure(fig4_)
            )






In [22]:
# if __name__ == '__main__':
#     app.run_server()

In [23]:
df1

,Unnamed: 0,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,0,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,2173,25,410,1738,2020-04-13 00:00:00+00:00
1,1,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,2272,27,410,1835,2020-04-14 00:00:00+00:00
2,2,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,2415,27,410,1978,2020-04-15 00:00:00+00:00
3,3,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,2506,34,410,2062,2020-04-16 00:00:00+00:00
4,4,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,2605,48,903,1654,2020-04-17 00:00:00+00:00
5,5,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,2783,50,903,1830,2020-04-18 00:00:00+00:00
6,6,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,3034,52,903,2079,2020-04-19 00:00:00+00:00
7,7,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,3158,54,903,2201,2020-04-20 00:00:00+00:00
8,8,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,3300,58,1055,2187,2020-04-21 00:00:00+00:00
9,9,South Africa,ZA,NaN,NaN,NaN,-30.56,22.94,3465,58,1055,2352,2020-04-22 00:00:00+00:00
